# Objetivo.
 
* Foi visto que o Target está desbalanceado, logo será visto o resultados dos modelos quando as classes ficaram equilibradas. 
 
* Obs: Foram feitos vários testes com diversos algoritmos para balancear as classes, e aquele que teve o melhor desempenho foi o TomekLinks.
 
 


#Funções

In [ ]:
def resultados(X_train,y_train,model):
  j= 0
  acerto_v = np.zeros(10)
  for linhas_treino, linhas_valid in kf.split(X_train):
    X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
    y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

    model = model.fit(X_t, y_t)
    prf = model.predict_proba(X_v)[:,1]
    prf_ = (prf > 0.42).astype(int)
    acerto_v[j] = f1_score(y_v, prf_)
    j = j + 1
  return np.mean(acerto_v)

def resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model,flag):
  
  if flag == 1:
    smote = TomekLinks()
    X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
    y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]
    X_t, y_t = smote.fit_resample(X_t, y_t)
    print('a')

  if flag == 0:
    smote = TomekLinks()
    X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
    y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]
    



  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  return prf






# Bibliotecas

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, f1_score
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from gokinjo import knn_kfold_extract
from gokinjo import knn_extract
from sklearn.preprocessing import LabelEncoder

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importando os dados

In [ ]:
def encoder_var(df,flag):
  if flag == 1:
    ind = df.columns[df.dtypes == 'object']
  if flag == 0:
     ind = df.columns[df.dtypes == 'category']

  for i in range(len(ind)):
    enc = LabelEncoder()
    inteiros = enc.fit_transform(df[ind[i]])
    df[ind[i]] = inteiros

  return df


def tratamento(df, flag):

  df['func_idade_log'] = np.log(df['func_idade'])
  df['contrato_horastrabalho_log'] = np.log(df['contrato_horastrabalho'] + 1)
  df['contrato_salario_log'] = np.log10(df['contrato_salario'])
  if flag == 1:
    df['turnover_apos_1_ano'] = df['turnover_apos_1_ano'].map({'NAO': 0, 'SIM': 1})
  
  df['func_deficiencia'] = df['func_deficiencia'].map({'NAO': 0, 'SIM': 1})
  df['func_sexo'] = df['func_sexo'].map({'FEMININO': 0, 'MASCULINO': 1})

  df = df.drop(columns = ['contrato_horastrabalho_log'])

  df = encoder_var(df,1)

  col2 = ['func_idade' ,'contrato_salario_log']
  df = df.drop(columns = col2)

  df['contrato_horastrabalho'].loc[df['contrato_horastrabalho'] == 0] =df['contrato_horastrabalho'].loc[df['contrato_horastrabalho'] > 0].mean()

  df['contrato_horastrabalho']= pd.cut(df["contrato_horastrabalho"], bins= 12)

  print(np.unique(df['contrato_horastrabalho']))

  #df['contrato_salario_sqrt']= df['contrato_salario']**(1/3)

  df = encoder_var(df,0)

  return df

In [ ]:
df = pd.read_csv('/content/drive/My Drive/competição flai nova/treino.csv',encoding = "UTF-8", sep = ";",decimal = ',',dayfirst = True)

In [ ]:
df_teste = pd.read_csv('/content/drive/My Drive/competição flai nova/teste.csv',encoding = "UTF-8", sep = ";",decimal = ',',dayfirst = True)

In [ ]:
df = tratamento(df,1)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


[Interval(0.957, 4.583, closed='right')
 Interval(4.583, 8.167, closed='right')
 Interval(8.167, 11.75, closed='right')
 Interval(11.75, 15.333, closed='right')
 Interval(15.333, 18.917, closed='right')
 Interval(18.917, 22.5, closed='right')
 Interval(22.5, 26.083, closed='right')
 Interval(26.083, 29.667, closed='right')
 Interval(29.667, 33.25, closed='right')
 Interval(33.25, 36.833, closed='right')
 Interval(36.833, 40.417, closed='right')
 Interval(40.417, 44.0, closed='right')]


In [ ]:
df_teste = tratamento(df_teste,0)

[Interval(0.957, 4.583, closed='right')
 Interval(4.583, 8.167, closed='right')
 Interval(8.167, 11.75, closed='right')
 Interval(11.75, 15.333, closed='right')
 Interval(15.333, 18.917, closed='right')
 Interval(18.917, 22.5, closed='right')
 Interval(22.5, 26.083, closed='right')
 Interval(26.083, 29.667, closed='right')
 Interval(29.667, 33.25, closed='right')
 Interval(33.25, 36.833, closed='right')
 Interval(36.833, 40.417, closed='right')
 Interval(40.417, 44.0, closed='right')]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Balaceamento de classe

### Xboost

**Resultado sem balancear as classes**

In [ ]:
X_train  = df.drop(columns='turnover_apos_1_ano')
y_train = df['turnover_apos_1_ano']

In [ ]:
lr = 0.027840476080879074
max_depth = 6
n_estimators = 669
colsample_bytree = 1
lamb             = 0.0009938152575176688
alpha            = 0.0
model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)

In [ ]:
kf = KFold(n_splits=10, random_state=5, shuffle=True)

In [ ]:
j= 0
acerto_v = np.zeros(10)
for linhas_treino, linhas_valid in kf.split(X_train):
  X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > 0.38).astype(int)
  acerto_v[j] = f1_score(y_v, prf_)
  j = j + 1
 

In [ ]:
print('F1:',np.mean(acerto_v),'Desvio padrão:',np.std(acerto_v),'erro:',np.std(acerto_v)/np.sqrt(10))

F1: 0.7998609908018577 Desvio padrão: 0.012525115347932141 erro: 0.00396078924557979


**Resultado com as classes balanceadas**

In [ ]:
acerto_v1 = np.zeros(10)
j= 0
smote = TomekLinks()
for linhas_treino, linhas_valid in kf.split(X_train):
    X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
    y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]
    X_t, y_t = smote.fit_resample(X_t, y_t)
    model = model.fit(X_t, y_t)
    prf = model.predict_proba(X_v)[:,1]
    prf_ = (prf > 0.38).astype(int)
    acerto_v1[j] = f1_score(y_v, prf_)
    j = j + 1
    
  
print('F1:',np.mean(acerto_v1),'Desvio padrão:',np.std(acerto_v1),'erro:',np.std(acerto_v1)/np.sqrt(10))
print('\n')

F1: 0.800809597106044 Desvio padrão: 0.013171973999301397 erro: 0.004165343911830955




### Stacking 

**Resultado sem balancear as classes**

In [ ]:
model1 = RandomForestClassifier(n_estimators = 100,max_features = 'sqrt',max_depth=12,random_state = 10)
model2 = XGBClassifier(colsample_bytree = 1, learning_rate = 0.011484853111591423,
                       max_depth = 5, n_estimators = 268,random_state=40)

clf    = MLPClassifier(hidden_layer_sizes =[261],activation='relu',solver='adam',alpha= 0.0006788795301189604,max_iter=500,random_state=40)
model3 = make_pipeline(MinMaxScaler(), clf)

clf    = MLPClassifier(hidden_layer_sizes =[1],activation='tanh',solver='lbfgs',alpha=0,max_iter=500,random_state=40)
model4 = make_pipeline(MinMaxScaler(), clf)

lr = 0.027840476080879074
max_depth = 6
n_estimators = 669
colsample_bytree = 1
lamb             = 0.0009938152575176688
alpha            = 0.0

model5 = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)








acertof1 = np.zeros(10)
acertol  = np.zeros(10)
second_level = np.zeros((X_train.shape[0], 5)) # dados de entrada do modelo 
j = 0
for linhas_treino, linhas_valid in kf.split(X_train):
  prf1 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model1,0)
  prf2 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model2,0)
  prf3 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model3,0)
  prf4 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model4,0)
  prf5 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model5,0)
  
  j =  j+1

  second_level[linhas_valid, 0] = prf1
  second_level[linhas_valid, 1] = prf2
  second_level[linhas_valid, 2] = prf3
  second_level[linhas_valid, 3] = prf4
  second_level[linhas_valid, 4] = prf5
 

In [ ]:
df_staking = pd.DataFrame(second_level,columns=['Fl', 'XGBM1', 'MLP1', 'MLP2','XGBM2'])

In [ ]:
lr = 0.04248763694738395
max_depth = 1
n_estimators = 1000
colsample_bytree = 1
reg_lambda = 4.222462269882505e-05
reg_alpha = 0.1

model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)

In [ ]:
X = df_staking[['XGBM1','XGBM2']]

In [ ]:
j= 0
acerto_v = np.zeros(10)
for linhas_treino, linhas_valid in kf.split(X):
  X_t, X_v = X.loc[linhas_treino], X.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > 0.40).astype(int)
  acerto_v[j] = f1_score(y_v, prf_)
  j = j + 1

In [ ]:
print('F1:',np.mean(acerto_v),'Desvio padrão:',np.std(acerto_v),'erro:',np.std(acerto_v)/np.sqrt(10))
print('\n')

F1: 0.7992239469925523 Desvio padrão: 0.012313593871562905 erro: 0.0038939002816429633




**Resultado com o balanceamento de classe**

In [ ]:
acertof1 = np.zeros(10)
acertol  = np.zeros(10)
second_level = np.zeros((X_train.shape[0], 5)) # dados de entrada do modelo 
j = 0
smote = TomekLinks()
for linhas_treino, linhas_valid in kf.split(X_train):
  prf1 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model1,1)
  prf2 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model2,1)
  prf3 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model3,1)
  prf4 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model4,1)
  prf5 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model5,1)
  
  j =  j+1

  second_level[linhas_valid, 0] = prf1
  second_level[linhas_valid, 1] = prf2
  second_level[linhas_valid, 2] = prf3
  second_level[linhas_valid, 3] = prf4
  second_level[linhas_valid, 4] = prf5

In [ ]:
df_staking = pd.DataFrame(second_level,columns=['Fl', 'XGBM1', 'MLP1', 'MLP2','XGBM2'])

In [ ]:
X = df_staking[['XGBM1','XGBM2']]

In [ ]:
lr = 0.04248763694738395
max_depth = 1
n_estimators = 1000
colsample_bytree = 1
reg_lambda = 4.222462269882505e-05
reg_alpha = 0.1

model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)

In [ ]:
j= 0
acerto_v = np.zeros(10)
for linhas_treino, linhas_valid in kf.split(X):
  X_t, X_v = X.loc[linhas_treino], X.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > 0.40).astype(int)
  acerto_v[j] = f1_score(y_v, prf_)
  j = j + 1

In [ ]:
print('F1:',np.mean(acerto_v),'Desvio padrão:',np.std(acerto_v),'erro:',np.std(acerto_v)/np.sqrt(10))
print('\n')

F1: 0.7996771316586271 Desvio padrão: 0.01316718709617692 erro: 0.004163830160149763




* Usando todos estimadores do primeiro nível

In [ ]:
X = df_staking[['XGBM2', 'XGBM1']]

In [ ]:
j= 0
acerto_v = np.zeros(10)
for linhas_treino, linhas_valid in kf.split(X):
  X_t, X_v = X.loc[linhas_treino], X.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > 0.40).astype(int)
  acerto_v[j] = f1_score(y_v, prf_)
  j = j + 1

In [ ]:
print('F1:',np.mean(acerto_v),'Desvio padrão:',np.std(acerto_v),'erro:',np.std(acerto_v)/np.sqrt(10))
print('\n')

F1: 0.7996771316586271 Desvio padrão: 0.01316718709617692 erro: 0.004163830160149763




# Conclusão:
 
* O equilíbrio das classes fez com que o modelo passasse de um f1 de 0.79986 Desvio padrão: 0.012525 erro: 0.00396 para 0.80080 Desvio padrão: 0.013171 erro: 0.00416.


# Competição.

###XGBM

In [ ]:
df = pd.read_csv('/content/drive/My Drive/competição flai nova/treino.csv',encoding = "UTF-8", sep = ";",decimal = ',',dayfirst = True)

In [ ]:
df_teste = pd.read_csv('/content/drive/My Drive/competição flai nova/teste.csv',encoding = "UTF-8", sep = ";",decimal = ',',dayfirst = True)

In [ ]:
df = tratamento(df,1)

[Interval(0.957, 4.583, closed='right')
 Interval(4.583, 8.167, closed='right')
 Interval(8.167, 11.75, closed='right')
 Interval(11.75, 15.333, closed='right')
 Interval(15.333, 18.917, closed='right')
 Interval(18.917, 22.5, closed='right')
 Interval(22.5, 26.083, closed='right')
 Interval(26.083, 29.667, closed='right')
 Interval(29.667, 33.25, closed='right')
 Interval(33.25, 36.833, closed='right')
 Interval(36.833, 40.417, closed='right')
 Interval(40.417, 44.0, closed='right')]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df_teste = tratamento(df_teste,0)

[Interval(0.957, 4.583, closed='right')
 Interval(4.583, 8.167, closed='right')
 Interval(8.167, 11.75, closed='right')
 Interval(11.75, 15.333, closed='right')
 Interval(15.333, 18.917, closed='right')
 Interval(18.917, 22.5, closed='right')
 Interval(22.5, 26.083, closed='right')
 Interval(26.083, 29.667, closed='right')
 Interval(29.667, 33.25, closed='right')
 Interval(33.25, 36.833, closed='right')
 Interval(36.833, 40.417, closed='right')
 Interval(40.417, 44.0, closed='right')]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
lr = 0.027840476080879074
max_depth = 6
n_estimators = 669
colsample_bytree = 1
lamb             = 0.0009938152575176688
alpha            = 0.0
model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)

In [ ]:
X_train  = df.drop(columns='turnover_apos_1_ano')
y_train = df['turnover_apos_1_ano']

In [ ]:
smote            = TomekLinks()
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [ ]:
model = model.fit(X_smote, y_smote)
prf = model.predict_proba(df_teste)[:,1]
prf_ = (prf > 0.38).astype(int)


In [ ]:
subm = pd.DataFrame()
subm['turnover_apos_1_ano'] = prf_

In [ ]:
subm['turnover_apos_1_ano'] = subm['turnover_apos_1_ano'].map({0:'NAO', 1: 'SIM'})

In [ ]:
subm

,turnover_apos_1_ano
0,SIM
1,NAO
2,NAO
3,NAO
4,NAO
...,...
14995,NAO
14996,SIM
14997,SIM
14998,NAO


In [ ]:
subm.to_csv('/content/drive/My Drive/competição flai nova/submissão1.csv', index = False)

* Com equilíbrio das classes o resultado na competição passou de 0.798 para 0.80124.


